In [137]:
import pandas as pd
from sklearn import tree
from sklearn import model_selection
from sklearn import ensemble

In [118]:
ghost_train=pd.read_csv("D:/Data/ghost_goblin/train.csv")

In [119]:
ghost_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 7 columns):
id               371 non-null int64
bone_length      371 non-null float64
rotting_flesh    371 non-null float64
hair_length      371 non-null float64
has_soul         371 non-null float64
color            371 non-null object
type             371 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 20.4+ KB


In [199]:
ghost_train.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost


In [198]:
ghost_train.describe()

,id,bone_length,rotting_flesh,hair_length,has_soul
count,371.000000,371.000000,371.000000,371.000000,371.000000
mean,443.676550,0.434160,0.506848,0.529114,0.471392
std,263.222489,0.132833,0.146358,0.169902,0.176129
min,0.000000,0.061032,0.095687,0.134600,0.009402
25%,205.500000,0.340006,0.414812,0.407428,0.348002
50%,458.000000,0.434891,0.501552,0.538642,0.466372
75%,678.500000,0.517223,0.603977,0.647244,0.600610
max,897.000000,0.817001,0.932466,1.000000,0.935721


In [120]:
ghost_train.corr()

,id,bone_length,rotting_flesh,hair_length,has_soul
id,1.000000,-0.062759,-0.007539,0.076475,0.100054
bone_length,-0.062759,1.000000,-0.041716,0.353881,0.381675
rotting_flesh,-0.007539,-0.041716,1.000000,-0.220353,-0.132051
hair_length,0.076475,0.353881,-0.220353,1.000000,0.474835
has_soul,0.100054,0.381675,-0.132051,0.474835,1.000000


In [224]:
import statistics 
statistics.stdev(ghost_train.bone_length)
statistics.stdev(ghost_train.rotting_flesh)
statistics.stdev(ghost_train.hair_length)
statistics.stdev(ghost_train.has_soul)

0.1761293396182109

In [168]:
x_train=pd.get_dummies(ghost_train.drop(['type','id'],1),columns=['color'])

In [47]:
def convert_soul(size):
    if(size >= 0.5): 
        return 1
    else: 
        return 0

In [114]:
x_train['soul1']=x_train.has_soul.map(convert_soul)
x_train=x_train.drop('has_soul',1)

In [169]:
x_train.head()

,bone_length,rotting_flesh,hair_length,has_soul,color_black,color_blood,color_blue,color_clear,color_green,color_white
0,0.354512,0.350839,0.465761,0.781142,0,0,0,1,0,0
1,0.575560,0.425868,0.531401,0.439899,0,0,0,0,1,0
2,0.467875,0.354330,0.811616,0.791225,1,0,0,0,0,0
3,0.776652,0.508723,0.636766,0.884464,1,0,0,0,0,0
4,0.566117,0.875862,0.418594,0.636438,0,0,0,0,1,0


In [17]:
y_train=ghost_train['type']

In [18]:
y_train.head()

0     Ghoul
1    Goblin
2     Ghoul
3     Ghoul
4     Ghost
Name: type, dtype: object

1. DecisionTree without Param

In [205]:
clf = tree.DecisionTreeClassifier(criterion="gini",random_state=0)
x = model_selection.cross_val_score(clf,x_train,y_train,cv=5)

In [206]:
x.mean()

0.6118622732321363

2. DecisionTree with GridSearchCV

In [207]:
dt = tree.DecisionTreeClassifier(random_state=0)
param_grid = {'max_depth':[5,8,10],'min_samples_split':[2,4,8,12,16],'criterion':['entropy','gini']}
dt_grid = model_selection.GridSearchCV(dt,param_grid)
dt_grid.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [5, 8, 10], 'min_samples_split': [2, 4, 8, 12, 16], 'criterion': ['entropy', 'gini']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [208]:
print(dt_grid.best_score_)
print(dt_grid.best_params_)

0.7142857142857143
{'criterion': 'gini', 'max_depth': 8, 'min_samples_split': 16}


3. RandomForest

In [214]:
rf_classifier = ensemble.RandomForestClassifier(random_state=0)
rf_grid = {'n_estimators':[10,20,30],'max_features':[4,5,6,7,8,9,10],'min_samples_split':[3,4,6]}
grid_rf_classifier = model_selection.GridSearchCV(rf_classifier,rf_grid,cv=10,n_jobs=3)
grid_rf_classifier.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'n_estimators': [10, 20, 30], 'max_features': [4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [3, 4, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [215]:
print(grid_rf_classifier.best_score_)
print(grid_rf_classifier.best_params_)

0.7169811320754716
{'max_features': 9, 'min_samples_split': 6, 'n_estimators': 30}


4. AdaBoost

In [216]:
ada_boost_estimator = ensemble.AdaBoostClassifier(random_state=0)
ada_boost_grid = {'n_estimators':[10,2,5,8,12,15,20],"learning_rate":[0.1,0.2,0.3,0.4]}
ada_boost_estimetor = model_selection.GridSearchCV(ada_boost_estimator,ada_boost_grid,cv=10,n_jobs=3)
ada_boost_estimetor.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=0),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'n_estimators': [10, 2, 5, 8, 12, 15, 20], 'learning_rate': [0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [217]:
print(ada_boost_estimetor.best_score_)
print(ada_boost_estimetor.best_params_)

0.7035040431266847
{'learning_rate': 0.4, 'n_estimators': 10}


In [2]:
from sklearn.cluster import KMeans

In [3]:
cluster1 = KMeans(n_clusters=3,max_iter=1000,random_state = 0)

In [4]:
cluster1.fit(x_train)

NameError: name 'x_train' is not defined

In [230]:
x_test = pd.read_csv("D:/Data/ghost_goblin/test.csv")

In [234]:
x_test=x_test.drop('id',1)

<bound method NDFrame.head of      bone_length  rotting_flesh  hair_length  has_soul  color
0       0.471774       0.387937     0.706087  0.698537  black
1       0.427332       0.645024     0.565558  0.451462  white
2       0.549602       0.491931     0.660387  0.449809  black
3       0.638095       0.682867     0.471409  0.356924  white
4       0.361762       0.583997     0.377256  0.276364  black
5       0.292890       0.505851     0.297927  0.106263  white
6       0.584456       0.551680     0.628330  0.499118  white
7       0.485525       0.459242     0.657735  0.565337   blue
8       0.471757       0.407916     0.465992  0.406608   blue
9       0.572953       0.316454     0.668388  0.640692  clear
10      0.320542       0.561671     0.340446  0.338964  white
11      0.413702       0.563031     0.822650  0.525445  clear
12      0.135615       0.513499     0.590764  0.549240  clear
13      0.457618       0.857810     0.414919  0.442086  clear
14      0.628138       0.439252     0.63

In [237]:
x_test.head()

,bone_length,rotting_flesh,hair_length,has_soul,color_black,color_blood,color_blue,color_clear,color_green,color_white
0,0.471774,0.387937,0.706087,0.698537,1,0,0,0,0,0
1,0.427332,0.645024,0.565558,0.451462,0,0,0,0,0,1
2,0.549602,0.491931,0.660387,0.449809,1,0,0,0,0,0
3,0.638095,0.682867,0.471409,0.356924,0,0,0,0,0,1
4,0.361762,0.583997,0.377256,0.276364,1,0,0,0,0,0


In [236]:
x_test=pd.get_dummies(x_test,columns=['color'])

In [238]:
cluster1.predict(x_test)

array([2, 1, 2, 1, 2, 1, 1, 2, 2, 0, 1, 0, 0, 0, 1, 2, 0, 2, 1, 0, 0, 2,
       0, 2, 0, 0, 2, 2, 2, 0, 1, 1, 2, 1, 0, 0, 2, 2, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 2, 2, 1, 1, 2, 2, 0, 1, 2, 2,
       1, 0, 1, 2, 0, 2, 2, 0, 0, 1, 0, 0, 0, 2, 2, 1, 1, 0, 2, 0, 1, 1,
       0, 2, 2, 1, 2, 2, 0, 0, 2, 2, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 2, 2, 1, 0, 2, 1, 2, 0, 0, 1, 2, 1, 1, 0, 2, 1, 0, 2, 1, 1,
       0, 2, 2, 1, 1, 0, 2, 0, 2, 0, 2, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1,
       2, 0, 1, 1, 0, 1, 0, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 1, 2, 2, 1, 0,
       1, 0, 1, 2, 1, 2, 2, 0, 1, 0, 0, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 0,
       0, 2, 1, 2, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 1, 0, 1, 2, 2, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 2, 1, 1, 1, 2, 2, 0, 2, 1, 0, 0, 1, 1,
       0, 2, 1, 1, 2, 0, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 0, 2, 0, 2,
       0, 1, 0, 0, 2, 0, 2, 1, 0, 2, 1, 0, 0, 0, 2, 0, 0, 1, 1, 0, 1, 2,
       1, 2, 2, 1, 2, 2, 1, 0, 2, 2, 1, 0, 2, 1, 0,